In [1]:
import pandas as pd
import io
import requests
import numpy as np
import random
import libpysal
import pysal
#import pysal.lib
import gpytorch
import torch
import math
from sklearn import preprocessing

C:\Users\kklemmer\AppData\Local\Continuum\anaconda3\lib\site-packages\pysal\model\spvcm\abstracts.py:10: UserWarning: The `dill` module is required to use the sqlite backend fully.
  from .sqlite import head_to_sql, start_sql


In [2]:
#Read data
data = pd.read_csv("grid_aug_housing_knn50.csv")
#Clean data
data_drop = data.head(18713) #Last column is broken!
data.drop(data_drop.tail(1).index,inplace=True)
data.drop(columns=["ocean_proximity"])
data = data[np.isfinite(data["total_bedrooms"])]
data = data[np.isfinite(data["total_rooms"])]
data = data[np.isfinite(data["housing_median_age"])]
data = data[np.isfinite(data["population"])]
data = data[np.isfinite(data["households"])]
data = data[np.isfinite(data["median_income"])]
data = data[np.isfinite(data["median_house_value"])]
#Create ID column
data["id"] = np.asarray(list(range(0,len(data["longitude"])))).reshape(-1,1)

In [3]:
#Create train/test lists by fold
d_train = {}
d_test = {}
for name in data.filter(regex="fold").columns:
    d_train[name] = data[data[name]==2]
    d_train[name] = d_train[name][d_train[name].columns.drop(list(d_train[name].filter(regex='fold')))] 
    d_test[name] = data[data[name]==1]
    d_test[name] = d_test[name][d_test[name].columns.drop(list(d_test[name].filter(regex='fold')))] 

In [4]:
random.seed(99)

In [5]:
def synth_point_gen(data,n):
    
    #Create running variable 
    i = 1
    #Create indicator column for synthetic data
    data["synth"] = 0
    #Reset index
    data = data.reset_index(drop=True)
    data.index = list(data.index)
    #Reset ID column
    data["id"] = np.asarray(list(range(0,len(data["longitude"])))).reshape(-1,1)
    
    #Create neighbourhood matrices
    kd = pysal.lib.cg.kdtree.KDTree(np.array(data[["longitude","latitude"]]))
    w_knn15 = pysal.lib.weights.KNN(kd, 15, ids=data["id"]) #KNN based weights
    w_knn20 = pysal.lib.weights.KNN(kd, 20, ids=data["id"]) 
    w_knn50 = pysal.lib.weights.KNN(kd, 50, ids=data["id"]) 
    w_mat = [w_knn15,w_knn20,w_knn50]

    #Create synthetic points until condition is met
    while i <= n:
        #Chose random datapoint
        random_sample = data.sample(n=1)
        #Chose random weihgtmatrix from set
        w = random.choice(w_mat)
        #Extract neighbourhood of random point
        temp_id = []
        for k in random_sample.index: 
                temp_id = np.unique(np.concatenate([temp_id,w.neighbors[k]]).ravel().astype(np.int32))
        #for l in temp_id: #Include second degree neighbors
        #       temp_id = np.unique(np.concatenate([temp_id,w.neighbors[l]]).ravel().astype(np.int32))
        temp = data.iloc[temp_id]
        #Bind random point with its neighbourhood
        random_sample = random_sample.append(temp)

        #Create new point using mean interpolation (for continuous variables) and random sampling (for discrete variables)
        new_point = pd.DataFrame(columns=data.columns)
        new_point = pd.DataFrame({"longitude":np.mean(random_sample["longitude"]),
                                  "latitude":np.mean(random_sample["latitude"]),
                                  "id":[np.max(data["id"])+1],
                                  "synth":[1]})
        #Bind with existing data
        data = data.append(new_point)
        #Increase counter
        i = i+1
    
    #Return data
    return data

In [6]:
test = synth_point_gen(d_train["fold2"],10000)
#test = test.reset_index()
test

C:\Users\kklemmer\AppData\Local\Continuum\anaconda3\lib\site-packages\pysal\lib\weights\weights.py:170: UserWarning: The weights matrix is not fully connected. There are 5 components
  warnings.warn("The weights matrix is not fully connected. There are %d components" % self.n_components)
C:\Users\kklemmer\AppData\Local\Continuum\anaconda3\lib\site-packages\pysal\lib\weights\weights.py:170: UserWarning: The weights matrix is not fully connected. There are 2 components
  warnings.warn("The weights matrix is not fully connected. There are %d components" % self.n_components)
C:\Users\kklemmer\AppData\Local\Continuum\anaconda3\lib\site-packages\pysal\lib\weights\weights.py:170: UserWarning: The weights matrix is not fully connected. There are 2 components
  warnings.warn("The weights matrix is not fully connected. There are %d components" % self.n_components)
C:\Users\kklemmer\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-c

,households,housing_median_age,id,latitude,longitude,median_house_value,median_income,ocean_proximity,population,synth,total_bedrooms,total_rooms
0,152.0,17.0,0,38.690000,-119.780000,117600.0,2.4500,INLAND,338.0,0,282.0,1364.0
1,196.0,15.0,1,38.720000,-119.930000,97900.0,2.2417,INLAND,573.0,0,465.0,2061.0
2,102.0,16.0,2,38.520000,-120.000000,140600.0,3.1500,INLAND,202.0,0,543.0,3045.0
3,11.0,33.0,3,38.000000,-122.390000,212500.0,4.1250,NEAR BAY,23.0,0,6.0,44.0
4,34.0,35.0,4,37.950000,-122.370000,81300.0,1.6023,NEAR BAY,100.0,0,45.0,215.0
5,268.0,32.0,5,37.950000,-122.370000,76400.0,0.9797,NEAR BAY,716.0,0,363.0,1298.0
6,25.0,36.0,6,37.980000,-122.410000,67500.0,1.4583,NEAR BAY,42.0,0,15.0,60.0
7,195.0,49.0,7,37.940000,-122.370000,71600.0,1.3167,NEAR BAY,599.0,0,229.0,969.0
8,239.0,40.0,8,37.940000,-122.370000,69100.0,1.0521,NEAR BAY,912.0,0,266.0,1064.0
9,749.0,45.0,9,37.930000,-122.370000,37900.0,1.7500,NEAR BAY,1798.0,0,756.0,3150.0


In [7]:
class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)

        # SKI requires a grid size hyperparameter. This util can help with that
        grid_size = gpytorch.utils.grid.choose_grid_size(train_x)

        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ConstantMean(), num_tasks=7
        )
        self.covar_module = gpytorch.kernels.MultitaskKernel(
            gpytorch.kernels.GridInterpolationKernel(
                gpytorch.kernels.RBFKernel(ard_num_dim=2,has_lengthscale=True), grid_size=grid_size, num_dims=2,
            ), num_tasks=7, rank=1
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)

In [8]:
final_data = test
final_data

,households,housing_median_age,id,latitude,longitude,median_house_value,median_income,ocean_proximity,population,synth,total_bedrooms,total_rooms
0,152.0,17.0,0,38.690000,-119.780000,117600.0,2.4500,INLAND,338.0,0,282.0,1364.0
1,196.0,15.0,1,38.720000,-119.930000,97900.0,2.2417,INLAND,573.0,0,465.0,2061.0
2,102.0,16.0,2,38.520000,-120.000000,140600.0,3.1500,INLAND,202.0,0,543.0,3045.0
3,11.0,33.0,3,38.000000,-122.390000,212500.0,4.1250,NEAR BAY,23.0,0,6.0,44.0
4,34.0,35.0,4,37.950000,-122.370000,81300.0,1.6023,NEAR BAY,100.0,0,45.0,215.0
5,268.0,32.0,5,37.950000,-122.370000,76400.0,0.9797,NEAR BAY,716.0,0,363.0,1298.0
6,25.0,36.0,6,37.980000,-122.410000,67500.0,1.4583,NEAR BAY,42.0,0,15.0,60.0
7,195.0,49.0,7,37.940000,-122.370000,71600.0,1.3167,NEAR BAY,599.0,0,229.0,969.0
8,239.0,40.0,8,37.940000,-122.370000,69100.0,1.0521,NEAR BAY,912.0,0,266.0,1064.0
9,749.0,45.0,9,37.930000,-122.370000,37900.0,1.7500,NEAR BAY,1798.0,0,756.0,3150.0


In [19]:
torch.cuda.empty_cache()

In [17]:
# GP INTERPOLATION (CAN BE COMMENTED OUT TO GET NA GRID)
#Get train and test data
train = final_data[final_data["synth"]==0]
test = final_data[final_data["synth"]==1]
#Convert to numpy
train_x = np.asarray(train[["longitude","latitude"]])
train_y = np.asarray(train[["housing_median_age","total_rooms","total_bedrooms","population","households","median_income","median_house_value"]])
test_x = np.asarray(test[["longitude","latitude"]])
#Scale input and output
scaler_train_y = preprocessing.StandardScaler().fit(train_y)
train_y = scaler_train_y.transform(train_y)
train_x = train_x / 100
test_x = test_x / 100
#Covert to pytorch tensor
train_x = torch.from_numpy(train_x).float()#.cuda()
train_y = torch.from_numpy(train_y).float()#.cuda()
test_x = torch.from_numpy(test_x).float()#.cuda()
#Train GP
likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=7)#.cuda()
model = MultitaskGPModel(train_x, train_y, likelihood)#.cuda()
# Find optimal model hyperparameters
model.train()
likelihood.train()
# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)#.cuda()
n_iter = 150
for i in range(n_iter):
    optimizer.zero_grad()
    output = model(train_x)
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f' % (i + 1, n_iter, loss.item()))
    optimizer.step()
likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=7)
model = MultitaskGPModel(train_x, train_y, likelihood)#.cuda()
# Set into eval mode
model.eval()
likelihood.eval()
# Make predictions
with torch.no_grad(), gpytorch.fast_pred_var():
    observed_pred = likelihood(model(test_x))
    # Get mean
    mean = observed_pred.mean
    # Get lower and upper confidence bounds
    lower, upper = observed_pred.confidence_region()
#Convert results back to GP
temp = test_x.cpu().numpy()
temp = temp * 100
test[["longitude","latitude"]] = temp
pred_y = mean.cpu().numpy()
pred_y = scaler_train_y.inverse_transform(pred_y)
test[["y","z"]] = pred_y     
#Create final dataframe
final_data = pd.concat([train, test])

Iter 1/150 - Loss: 19789.000


KeyboardInterrupt: 

In [11]:
train_x = np.asarray(data[["longitude","latitude"]])
train_y = np.asarray(data[["housing_median_age","total_rooms","total_bedrooms","population","households","median_income","median_house_value"]])

In [ ]:
from sklearn import preprocessing
train_x = train_x / 100
scaler_train_y = preprocessing.MinMaxScaler().fit_transform(train_y)
train_y = scaler_train_y.transform(train_y)

In [ ]:
train_x = torch.from_numpy(train_x).float()
train_y = torch.from_numpy(train_y).float()

In [ ]:
#train_x = torch.tensor(data[["longitude","latitude"]].values).float()
#train_y = torch.tensor(data[["housing_median_age","total_rooms","total_bedrooms","population","households","median_income","median_house_value"]].values).float()

In [ ]:
train_y.shape

In [ ]:
class MultitaskGPModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(MultitaskGPModel, self).__init__(train_x, train_y, likelihood)

        # SKI requires a grid size hyperparameter. This util can help with that
        grid_size = gpytorch.utils.grid.choose_grid_size(train_x)

        self.mean_module = gpytorch.means.MultitaskMean(
            gpytorch.means.ConstantMean(), num_tasks=7
        )
        self.covar_module = gpytorch.kernels.MultitaskKernel(
            gpytorch.kernels.GridInterpolationKernel(
                gpytorch.kernels.RBFKernel(ard_num_dim=2,has_lengthscale=True), grid_size=grid_size, num_dims=2,
            ), num_tasks=7, rank=1
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultitaskMultivariateNormal(mean_x, covar_x)


likelihood = gpytorch.likelihoods.MultitaskGaussianLikelihood(num_tasks=7)
model = MultitaskGPModel(train_x, train_y, likelihood)

In [ ]:
# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

n_iter = 100
for i in range(n_iter):
    optimizer.zero_grad()
    output = model(train_x)
    loss = -mll(output, train_y)
    loss.backward()
    print('Iter %d/%d - Loss: %.3f' % (i + 1, n_iter, loss.item()))
    optimizer.step()

In [ ]:
train_y[-1:,:]

In [ ]:
train_x[-1:,:]

In [ ]:
#mll.likelihood(output).log_prob(train_y).size()

In [ ]:
# Set into eval mode
model.eval()
likelihood.eval()

In [ ]:
# Make predictions
with torch.no_grad(), gpytorch.fast_pred_var():
    #test_x = torch.linspace(0, 1, 51)
    observed_pred = likelihood(model(train_x))
    # Get mean
    mean = observed_pred.mean
    # Get lower and upper confidence bounds
    lower, upper = observed_pred.confidence_region()

In [ ]:
mean

In [ ]:
mean[:1,:]

In [ ]:
train_y[:1,:]

In [ ]:
list(data[["housing_median_age","total_rooms","total_bedrooms","population","households","median_income","median_house_value"]])

In [ ]:
np.log10(710414720)